In [ ]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

In [ ]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "enrich" # the state this transform runs in
config_name = "pending_enrichment" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
input_pharma = "sun"
input_brand = "ilumya"
input_state = "ingest"
input_name = "symphony_health_association_ingest_column_mapping"
input_branch = "sun-extract-validation" # if None, input_branch is automagically set to your working branch

In [ ]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
transform_id = ids[0]
run_id = ids[1]

In [ ]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::[transform name here]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

import pandas as pd
pd.options.display.max_rows=999
pd.options.display.max_columns=999

In [ ]:
""" 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>

e.g.

class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''

    status_type: str
    id_cols: list
    ambig_list: list
    
    
transform = Transform()

In [ ]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

(clear out and replace with your description)

### Transformation

In [ ]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch, state=input_state, parent=input_pharma, child=input_brand, dataset=input_name)
run_filter = []
run_filter.append(dict(partition="__metadata_run_id", comparison="==", values=[4]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!
df = input_contract.fetch(filters=run_filter)

import pandas as pd
pd.options.display.max_columns=999

df.status_date = df.status_date.str[:8].astype(str)
df.ref_date = df.ref_date.str[:8].astype(str)

df.status_date = pd.to_datetime(df.status_date, infer_datetime_format=True, errors='coerce')
df.ref_date = pd.to_datetime(df.ref_date, infer_datetime_format=True, errors='coerce')

In [ ]:
df = df.sort_values(['msa_patient_id', 'pharm_code', 'medication', 'status_date'],ascending=[True, True, True, True])

In [ ]:
bvpa_df = df[df.status_code == 'PENDING']

In [204]:
df.status_code.unique()

array(['PENDING', 'CANCELLED', 'ACTIVE', 'DISCONTINUED', 'DENIED',
       'DISCONTTINUED'], dtype=object)

In [205]:
df.sub_status.unique()

array(['PATIENT CONTACT', 'PATIENT RESPONSE', 'OTHER', 'BENEFITS', 'PA',
       'NEW', 'SHIPMENT', 'INS OTHER', 'INSURANCE DENIED', 'PRESCRIBER',
       'READY', 'INSURANCE HOLD', 'PATIENT END', 'HOLD RTS',
       'INSURANCE OON', 'PRESCRIBER HOLD', 'APPEAL', 'PRESCRIBER END',
       'PATIENT FINANCIAL', 'PRESCRIBERHOLD', 'TRANSFER SP', 'FOUNDATION',
       'HOLD OTHER', 'PATIENT DECEASED', 'INFORMATION', 'PATIENT HOLD',
       'INVENTORY HOLD', 'TRANSFER HUB', 'DELAY', 'FORMULARY',
       'ALT THERAPY', 'SERVICES END', 'INSURANCE OTHER', 'TRANSER SP',
       'THERAPY COMPLETE', 'PT HOLD', 'P05'], dtype=object)

In [203]:
df[df.status_code == 'PENDING'].sub_status.unique()

array(['PATIENT CONTACT', 'OTHER', 'BENEFITS', 'PA', 'NEW',
       'PATIENT RESPONSE', 'PRESCRIBER', 'PRESCRIBER HOLD', 'APPEAL',
       'PATIENT FINANCIAL', 'PRESCRIBERHOLD', 'FOUNDATION', 'INFORMATION',
       'PATIENT HOLD', 'INVENTORY HOLD', 'DELAY', 'PRESCRIBER END', 'P05'],
      dtype=object)

In [ ]:
min_bvpa = bvpa_df.groupby(['msa_patient_id','pharm_code','medication']).status_date.min().reset_index(drop=False).rename(columns={'status_date':'min_bvpa_date'})
max_bvpa = bvpa_df.groupby(['msa_patient_id','pharm_code','medication']).status_date.max().reset_index(drop=False).rename(columns={'status_date':'max_bvpa_date'})

In [201]:
df = pd.merge(df,min_bvpa,on=['msa_patient_id','pharm_code','medication'])
df = pd.merge(df,max_bvpa,on=['msa_patient_id','pharm_code','medication'])

In [202]:
df.head()

,rec_date,pharm_code,pharm_npi,transtype,pharm_transaction_id,trans_seq,ref_source,ref_date,program_id,pharmacy_id,pat_last_name,pat_first_name,pat_dob,pat_gender,pat_addr1,pat_addr2,pat_city,pat_state,pat_zip,dx1_code,dx2_code,status_date,status_code,sub_status,pres_last_name,pres_first_name,pres_addr1,pres_addr2,pres_city,pres_state,pres_zip,pres_phone,pres_npi,pres_dea,facility_name,rxdate,rxnumber,rxrefills,rxfill,refill_remaining,prev_disp,rx_ndc_number,medication,quantity,day_supply,ship_date,ship_carrier,shiptracking_num,ship_location,ship_address,ship_city,ship_state,ship_zip,has_medical,primary_coverage_type,primary_payer_name,primary_payer_type,secondary_coverage_type,secondary_payer_name,secondary_payer_type,plan_paid_amt,pat_copay,copay_assist_amount,oth_payer_amt,xfer_pharmname,msa_patient_id,msa_patient_bmap,__metadata_run_timestamp,__metadata_app_version,__metadata_output_contract,__metadata_transform_timestamp,__metadata_run_id,min_bvpa_date,max_bvpa_date
0,20190325 23:00:00,CVS,1518948413,COM,901177673820190325000000,0,DIRECT,2019-03-15,None,9011776738,None,None,None,M,None,None,None,None,11,L40.0,None,2019-03-25,PENDING,PATIENT CONTACT,COHEN,RUSSELL,258 MERRICK RD,None,OCEANSIDE,NY,11572,5167660345,1538167242,BC1293911,None,20190315,84604413,0,None,0,None,47335017795,ILUMYA SD PFS,2,28,None,None,None,None,None,None,None,None,None,MEDICAL,None,None,None,None,None,None,None,None,None,None,2120006,VVVVV,2019-06-26 15:28:20,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-06-26 15:33:32,4,2019-03-25,2019-03-25
1,20190405 23:00:00,CVS,1518948413,COM,901177673820190405000000,0,DIRECT,2019-03-15,None,9011776738,None,None,None,M,None,None,None,None,11,L40.0,None,2019-04-05,CANCELLED,PATIENT RESPONSE,COHEN,RUSSELL,258 MERRICK RD,None,OCEANSIDE,NY,11572,5167660345,1538167242,BC1293911,None,20190315,84604413,0,None,0,None,47335017795,ILUMYA SD PFS,2,28,None,None,None,None,None,None,None,None,None,MEDICAL,None,None,None,None,None,None,None,None,None,None,2120006,VVVVV,2019-06-26 15:28:20,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-06-26 15:33:32,4,2019-03-25,2019-03-25
2,2019010406:34:02,BRV,1720364532,COM,BRIOVARX_20190104_118503541,0,DIRECT,2019-01-03,None,422180940,None,None,None,F,None,None,None,None,94,None,None,2019-01-03,PENDING,OTHER,GREENBERG,ROBERT,5201 NORRIS CANYON RD STE 1,None,SAN RAMON,CA,94583,9252771300,1639195316,None,None,20181019,None,01,00,None,None,47335017795,ILUMYA 100MG/ML PFS INJ,1,84,None,None,None,None,None,None,None,None,Y,MEDICAL,None,OTHER,None,None,None,None,None,None,None,None,2120009,VVVVV,2019-06-26 15:28:20,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-06-26 15:33:31,4,2019-01-03,2019-03-05
3,2019010506:34:05,BRV,1720364532,COM,BRIOVARX_20190105_118503541,0,DIRECT,2019-01-03,None,422180940,None,None,None,F,None,None,None,None,94,None,None,2019-01-04,PENDING,BENEFITS,GREENBERG,ROBERT,5201 NORRIS CANYON RD STE 1,None,SAN RAMON,CA,94583,9252771300,1639195316,None,None,20181019,None,01,00,None,None,47335017795,ILUMYA 100MG/ML PFS INJ,1,84,None,None,None,None,None,None,None,None,N,PHARMACY,UHC E AND I,COMMERCIAL,None,None,None,None,None,None,None,None,2120009,VVVVV,2019-06-26 15:28:20,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-06-26 15:33:31,4,2019-01-03,2019-03-05
4,2019030206:33:54,BRV,1720364532,COM,BRIOVARX_20190302_118503541,0,HUB,2019-01-03,1304154,422180940,None,None,None,F,None,None,None,None,94,None,None,2019-03-01,PENDING,BENEFITS,GREENBERG,ROBERT,5201 NORRIS CANYON RD STE 1,None,SAN RAMON,CA,94583,9252771300,1639195316,None,None,20181019,None,01,00,None,None,47335017795,ILUMYA 100MG/ML PFS INJ,1,84,None,None,None,None,None,None,None,None,N,PHARMACY,UHC E AND I,COMMERCIAL,None,None,None,None,None,None,None,None,2120009,VVVVV,2019-06-26 15:28:20,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-06-26 15:33:31,4,2019-01-03,2019-03-05


In [ ]:
bvpa_min_max = pd.merge(min_bvpa,max_bvpa)

In [199]:
bvpa_dict = bvpa_df[bvpa_df.sub_status == 'OTHER'][['pharm_code','medication','msa_patient_id']].drop_duplicates().dropna().reset_index(drop=True).to_dict(orient='index')

In [ ]:
id_list = []
max_shape = 0
for i in bvpa_dict.keys():
    curr_shape = df[(df.pharm_code == bvpa_dict[i]['pharm_code']) & (df.medication == bvpa_dict[i]['medication']) & (df.msa_patient_id == bvpa_dict[i]['msa_patient_id'])].shape[0]
    if curr_shape > 10:
        id_list.append(i)

In [ ]:
def pending_enrich(df, ambig_list, status_col, id_col_list, status_type):
    bvpa_df = df[df[status_col] == status_type]

In [ ]:
df = df.sort_values(['msa_patient_id', 'pharm_code', 'medication', 'status_date'],ascending=[True, True, True, True])

df.head()

In [ ]:
pj_dict = (
    df[['pharm_npi','medication']]
    .drop_duplicates()
    .dropna()
    .reset_index(drop=True)
    .to_dict(orient='index')
)

In [ ]:
def return_df(df,dct,key):
    col_set_1,col_set_2 = dct[key].items()
    return df[(df[col_set_1[0]] == col_set_1[1]) & (df[col_set_2[0]] == col_set_2[1]) & (df['status_code'] == 'PENDING')].shape

In [ ]:
df = df.drop(labels=['pat_last_name', 'pat_first_name', 'pat_dob', 'pat_gender', 'pat_addr1', 'pat_addr2', 'pat_city', 'pat_state', 'pat_zip', 'dx1_code', 'dx2_code','pres_last_name',
                     'pres_first_name', 'pres_addr1', 'pres_addr2', 'pres_city', 'pres_state', 'pres_zip', 'pres_phone', 'pres_npi', 'pres_dea', 'facility_name', 'rxdate', 'rxnumber',
                     'rxrefills', 'rxfill', 'refill_remaining', 'prev_disp', 'rx_ndc_number'],axis=1)

In [ ]:
by=[patient, pharm, product, status_date], ascending=[True, True, True, True]

In [ ]:
import unittest

def shape_status(final_dataframe,df):
    return df.shape == final_dataframe.shape

def substatus_cleaned(final_dataframe,sub_col,sub_status):
    return final_dataframe[final_dataframe[sub_col] == sub_status] == 0

class TestNotebook(unittest.TestCase):
    
    def test_shape_status(self):
        self.assertEqual(shape_status(final_dataframe,df),True)
        
    def test_substatus_cleaned(final_dataframe,'sub_status','OTHER')
    
unittest.main(argv=[''],verbosity=2,exit=False)

In [ ]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe

### Publish

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session)
session.close()